In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Replace with your actual path
file_path = '/content/drive/MyDrive/pincode_summary.csv'
df = pd.read_csv(file_path)

df.head()


,Pincode,Total_Deliveries,Successes,Failures,Delays,Avg_Delivery_Hours,Most_Common_Reason,Latitude,Longitude,Success Rate (%),Failure Rate (%),Delay Rate (%)
0,422001,375,269,37,69,48.576000,Weather,19.9975,73.7898,71.73,9.87,18.40
1,422002,377,261,51,65,47.745358,Traffic,20.0020,73.7900,69.23,13.53,17.24
2,422003,388,285,37,66,47.443299,Weather,20.0050,73.7950,73.45,9.54,17.01
3,422004,347,244,31,72,49.244957,Customer Unavailable,20.0100,73.8000,70.32,8.93,20.75
4,440001,366,250,37,79,49.704918,Weather,21.1458,79.0882,68.31,10.11,21.58


In [ ]:
df_cluster = df[df['Failure Rate (%)'] > 10].copy()


In [ ]:
from sklearn.cluster import KMeans

coords = df_cluster[['Latitude', 'Longitude']]

# Set number of clusters to min(4, number of samples)
k = min(4, len(coords))

# Only cluster if enough points
if k >= 2:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    df_cluster['Cluster'] = kmeans.fit_predict(coords)
    centroids = kmeans.cluster_centers_
else:
    print("❌ Not enough high-failure pincodes to cluster. Try reducing filter threshold.")


In [ ]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=[20.5, 78.9], zoom_start=7, tiles='CartoDB positron')
colors = ['red', 'green', 'blue', 'purple', 'orange']

# Plot pincode clusters
for _, row in df_cluster.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=7,
        color=colors[row['Cluster']],
        fill=True,
        fill_opacity=0.6,
        popup=f"Pincode: {row['Pincode']}<br>Failure Rate: {row['Failure Rate (%)']}%"
    ).add_to(m)

# Plot centroid markers (recommended hubs)
for i, (lat, lon) in enumerate(centroids):
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='black', icon='home'),
        popup=f"📦 Suggested Hub #{i+1}"
    ).add_to(m)

# Save and download map
m.save("hub_clusters_map.html")
from google.colab import files
files.download("hub_clusters_map.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>